In [1]:
pip install pyspark kubernetes pyyaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 5.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.1/223.1 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 19.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
from kubernetes import client, config
import yaml

# Load kubeconfig
config.load_incluster_config()

# Define SparkApplication as YAML string
spark_app_yaml = """
apiVersion: sparkoperator.k8s.io/v1beta2
kind: SparkApplication
metadata:
  name: spark-pi
  namespace: default
spec:
  type: Scala
  mode: cluster
  image: docker.io/library/spark:4.0.0
  imagePullPolicy: IfNotPresent
  mainClass: org.apache.spark.examples.SparkPi
  mainApplicationFile: local:///opt/spark/examples/jars/spark-examples.jar
  arguments:
  - "5000"
  sparkVersion: 4.0.0
  sparkConf:
    spark.kubernetes.executor.deleteOnTermination: "false" # Keep the executor pods after completion to inspect the logs
  driver:
    labels:
      version: 4.0.0
    cores: 1
    memory: 512m
    serviceAccount: spark-operator-spark
    securityContext:
      capabilities:
        drop:
        - ALL
      runAsGroup: 185
      runAsUser: 185
      runAsNonRoot: true
      allowPrivilegeEscalation: false
      seccompProfile:
        type: RuntimeDefault
  executor:
    labels:
      version: 4.0.0
    instances: 1
    cores: 1
    memory: 512m
    securityContext:
      capabilities:
        drop:
        - ALL
      runAsGroup: 185
      runAsUser: 185
      runAsNonRoot: true
      allowPrivilegeEscalation: false
      seccompProfile:
        type: RuntimeDefault
"""

# Parse YAML to Python dict
spark_app = yaml.safe_load(spark_app_yaml)

# Create the SparkApplication
api = client.CustomObjectsApi()
api.create_namespaced_custom_object(
    group="sparkoperator.k8s.io",
    version="v1beta2",
    namespace="default",
    plural="sparkapplications",
    body=spark_app
)

{'apiVersion': 'sparkoperator.k8s.io/v1beta2',
 'kind': 'SparkApplication',
 'metadata': {'creationTimestamp': '2025-11-09T03:54:48Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'sparkoperator.k8s.io/v1beta2',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:spec': {'.': {},
      'f:arguments': {},
      'f:driver': {'.': {},
       'f:cores': {},
       'f:labels': {'.': {}, 'f:version': {}},
       'f:memory': {},
       'f:securityContext': {'.': {},
        'f:allowPrivilegeEscalation': {},
        'f:capabilities': {'.': {}, 'f:drop': {}},
        'f:runAsGroup': {},
        'f:runAsNonRoot': {},
        'f:runAsUser': {},
        'f:seccompProfile': {'.': {}, 'f:type': {}}},
       'f:serviceAccount': {}},
      'f:executor': {'.': {},
       'f:cores': {},
       'f:instances': {},
       'f:labels': {'.': {}, 'f:version': {}},
       'f:memory': {},
       'f:securityContext': {'.': {},
        'f:allowPrivilegeEscalation': {},
        'f:capabilities': {'.': {}, '